<a href="https://colab.research.google.com/github/Murilosarto/SQL-Study/blob/main/Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
#Importação da Biblioteca
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import train_test_split,TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import holidays
import plotly.graph_objects as go
from lightgbm import LGBMRegressor
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet
import time
from sklearn.ensemble import RandomForestRegressor
import itertools
from skforecast.recursive import ForecasterRecursive
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import TimeSeriesFold

In [2]:
df = pd.read_csv('/content/Check.csv',sep=';')

In [3]:
df = df[df['NAID'] != 'BRU70']

In [4]:
df['Canal'] = df.apply(lambda x: 'Digital' if x['NAID'] == 'BRA02' else 'Own Stores' if
                       x['NAID'] == 'BR996' else 'Retail',axis=1)

In [5]:
df['TOP_CORRIDOR'] = df['CORRIDOR'].apply(lambda x: 'OTHERS' if x != 'AR' and x != 'CO' and x != 'BO' and
                                          x != 'MZ' and x != 'US' and x != 'PT' and
                                          x != 'CL' and x != 'PY' and x != 'PE' and x != 'HT' else x)

In [18]:
df = df.groupby(['DTEE'])['TRANSACTIONS_TOTAL'].sum().reset_index()

In [11]:
df['DTEE'] = df['DTEE'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))

In [22]:
df['Year'] = df['DTEE'].dt.year
df['Month'] = df['DTEE'].dt.month
df['Day'] = df['DTEE'].dt.day
df['DayOfWeek'] = df['DTEE'].dt.weekday

In [23]:
br_holidays = holidays.Brazil(years=[2020,2022,2023,2024,2025])

df['IsHoliday'] = df['DTEE'].apply(lambda x: 1 if x in br_holidays else 0)
df['NewYearImpact'] = df['DTEE'].apply(lambda x: 1 if x.strftime('%m-%d') in ['12-31','01-01'] else 0)
df['IsWeekday'] = df['DayOfWeek'].apply(lambda x: 1 if x<5 else 0)

df = pd.get_dummies(df, columns=['Month','Day', 'DayOfWeek'],drop_first=True)

In [24]:
lags = [1,7,30]
for lag in lags:
  df[f'Lag_{lag}'] = df['TRANSACTIONS_TOTAL'].shift(lag)

In [25]:
df['MovingAvg_7'] = df['TRANSACTIONS_TOTAL'].rolling(window=7).mean()
df['MovingAvg_30'] = df['TRANSACTIONS_TOTAL'].rolling(window=30).mean()

In [26]:
df.dropna(inplace=True)

In [49]:
x = df.drop(columns=['TRANSACTIONS_TOTAL','DTEE'])
y = df['TRANSACTIONS_TOTAL']

In [50]:
train_size = int(len(df)*0.8)
x_train,x_test = x.iloc[:train_size],x.iloc[train_size:]
y_train,y_test = y.iloc[:train_size],y.iloc[train_size:]

In [51]:
from sklearn.ensemble import RandomForestRegressor

In [52]:
models = {
    'RandomForest': RandomForestRegressor(n_estimators=200,random_state=42),
    'XGBoost': XGBRegressor(n_estimators=200,learning_rate=0.1,objective='reg:squarederror')
}

In [ ]:
for name,model in models.items():
  model.fit(x_train,y_train)
  predictions = model.predict(x_test)

  rmse = np.sqrt(mean_squared_error(y_test,predictions))
  mae = mean_absolute_error(y_test,predictions)

In [15]:
best_model = XGBRegressor(n_estimators=200,learning_rate=0.1,objective='reg:squarederror')
best_model.fit(x_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [16]:
y_pred = best_model.predict(x_test)

In [17]:
df = df.sort_values('DTEE')

In [18]:
fig = go.Figure()
trace1 = go.Scatter(x=df['DTEE'].iloc[train_size:], y=y_test, name="test", mode="lines")
trace2 = go.Scatter(x=df['DTEE'].iloc[train_size:], y=y_pred, name="prediction", mode="lines")
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.update_layout(
    title="Real value vs predicted in test data",
    xaxis_title="Date time",
    yaxis_title="TRANSACTIONS",
    width=750,
    height=350,
    margin=dict(l=20, r=20, t=35, b=20),
    legend=dict(orientation="h", yanchor="top", y=1.1, xanchor="left", x=0.001)
)
fig.show(renderer='colab')

In [19]:
future_df = pd.date_range(start="2025-01-01",periods=31,freq='D')

future_df = pd.DataFrame({'Date':future_df})
future_df['Year'] = future_df['Date'].dt.year
future_df['Month'] = future_df['Date'].dt.month
future_df['Day'] = future_df['Date'].dt.day
future_df['DayOfWeek'] = future_df['Date'].dt.weekday
future_df['IsHoliday'] = future_df['Date'].apply(lambda x: 1 if x in br_holidays else 0)
future_df['IsNewYearOrEnd'] = future_df['Date'].apply(lambda x: 1 if x.strftime('%m-%d') in ['12-31','01-01'] else 0)
future_df['IsWeekday'] = future_df['DayOfWeek'].apply(lambda x: 1 if x<5 else 0)

In [20]:
future_df = pd.get_dummies(future_df, columns=['Month','Day', 'DayOfWeek'],drop_first=True)

In [21]:
for lag in lags:
  future_df[f'Lag_{lag}'] = y.iloc[-lag]

future_df['MovingAvg_7'] = y.rolling(window=7).mean().iloc[-1]
future_df['MovingAvg_30'] = y.rolling(window=30).mean().iloc[-1]

future_df.fillna(0,inplace=True)

x_future = future_df.drop(columns=['Date'])

x_future,_ = x_future.align(x_train,join='left',axis=1, fill_value=0)

In [22]:
x_future = x_future.reindex(columns=x_train.columns,fill_value=0)

In [23]:
missing_in_future = set(x_train.columns)-set(x_future.columns)
extra_in_future = set(x_future.columns)-set(x_train.columns)

print(f'Missing in future: {missing_in_future}')
print(f'Extra in future: {extra_in_future}')

Missing in future: set()
Extra in future: set()


In [24]:
future_df['Predicted_transactions']=best_model.predict(x_future)

In [25]:
future_df['Predicted_transactions'].sum()

182855.25

In [9]:
# Criar um DataFrame para armazenar todas as previsões
all_predictions = []

# Obter listas únicas de canais e corredores
canais = df['Canal'].unique()
corredores = df['TOP_CORRIDOR'].unique()

# Iterar sobre cada combinação de canal e corredor
for canal in canais:
   for corredor in corredores:
       print(f"Treinando modelo para Canal: {canal} | Corredor: {corredor}")
       # Filtrar os dados para a combinação específica
       df_filtered = df[(df['Canal'] == canal) & (df['TOP_CORRIDOR'] == corredor)].copy()

       df_filtered = df_filtered.groupby(['DTEE'])['TRANSACTIONS_TOTAL'].sum().reset_index()

       df_filtered['DTEE'] = df_filtered['DTEE'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))
       df_filtered['Year'] = df_filtered['DTEE'].dt.year
       df_filtered['Month'] = df_filtered['DTEE'].dt.month
       df_filtered['Day'] = df_filtered['DTEE'].dt.day
       df_filtered['DayOfWeek'] = df_filtered['DTEE'].dt.weekday
       df_filtered  = df_filtered.sort_values('DTEE')

       br_holidays = holidays.Brazil(years=[2020,2022,2023,2024,2025])

       df_filtered['IsHoliday'] = df_filtered['DTEE'].apply(lambda x: 1 if x in br_holidays else 0)
       df_filtered['NewYearImpact'] = df_filtered['DTEE'].apply(lambda x: 1 if x.strftime('%m-%d') in ['12-31','01-01'] else 0)
       df_filtered['IsWeekday'] = df_filtered['DayOfWeek'].apply(lambda x: 1 if x<5 else 0)

       df_filtered = pd.get_dummies(df_filtered, columns=['Month','Day', 'DayOfWeek'],drop_first=True)

       lags = [1,7,30]
       for lag in lags:
        df_filtered[f'Lag_{lag}'] = df_filtered['TRANSACTIONS_TOTAL'].shift(lag)

       df_filtered['MovingAvg_7'] = df_filtered['TRANSACTIONS_TOTAL'].rolling(window=7).mean()
       df_filtered['MovingAvg_30'] = df_filtered['TRANSACTIONS_TOTAL'].rolling(window=30).mean()
       df_filtered.dropna(inplace=True)

       x = df_filtered.drop(columns=['TRANSACTIONS_TOTAL','DTEE'])
       y = df_filtered['TRANSACTIONS_TOTAL']

       train_size = int(len(df_filtered)*0.8)
       x_train,x_test = x.iloc[:train_size],x.iloc[train_size:]
       y_train,y_test = y.iloc[:train_size],y.iloc[train_size:]

       best_model = XGBRegressor(n_estimators=200,learning_rate=0.1,objective='reg:squarederror')
       best_model.fit(x_train,y_train)

       future_df = pd.date_range(start="2025-01-01",periods=365,freq='D')
       future_df = pd.DataFrame({'Date':future_df})
       future_df['Year'] = future_df['Date'].dt.year
       future_df['Month'] = future_df['Date'].dt.month
       future_df['Day'] = future_df['Date'].dt.day
       future_df['DayOfWeek'] = future_df['Date'].dt.weekday
       future_df['IsHoliday'] = future_df['Date'].apply(lambda x: 1 if x in br_holidays else 0)
       future_df['IsNewYearOrEnd'] = future_df['Date'].apply(lambda x: 1 if x.strftime('%m-%d') in ['12-31','01-01'] else 0)
       future_df['IsWeekday'] = future_df['DayOfWeek'].apply(lambda x: 1 if x<5 else 0)

       future_df = pd.get_dummies(future_df, columns=['Month','Day', 'DayOfWeek'],drop_first=True)

       for lag in lags:
        future_df[f'Lag_{lag}'] = y.iloc[-lag]

       future_df['MovingAvg_7'] = y.rolling(window=7).mean().iloc[-1]
       future_df['MovingAvg_30'] = y.rolling(window=30).mean().iloc[-1]

       future_df.fillna(0,inplace=True)

       x_future = future_df.drop(columns=['Date'])

       x_future,_ = x_future.align(x_train,join='left',axis=1, fill_value=0)

       x_future = x_future.reindex(columns=x_train.columns,fill_value=0)

       missing_in_future = set(x_train.columns)-set(x_future.columns)
       extra_in_future = set(x_future.columns)-set(x_train.columns)

       print(f'Missing in future: {missing_in_future}')
       print(f'Extra in future: {extra_in_future}')

       future_df['Predicted_transactions']=best_model.predict(x_future)
       future_df['Canal'] = canal
       future_df['TOP_CORRIDOR'] = corredor

       all_predictions.append(future_df)

Treinando modelo para Canal: Own Stores | Corredor: OTHERS
Missing in future: set()
Extra in future: set()
Treinando modelo para Canal: Own Stores | Corredor: HT
Missing in future: set()
Extra in future: set()
Treinando modelo para Canal: Own Stores | Corredor: US
Missing in future: set()
Extra in future: set()
Treinando modelo para Canal: Own Stores | Corredor: CO
Missing in future: set()
Extra in future: set()
Treinando modelo para Canal: Own Stores | Corredor: PE
Missing in future: set()
Extra in future: set()
Treinando modelo para Canal: Own Stores | Corredor: BO
Missing in future: set()
Extra in future: set()
Treinando modelo para Canal: Own Stores | Corredor: CL
Missing in future: set()
Extra in future: set()
Treinando modelo para Canal: Own Stores | Corredor: AR
Missing in future: set()
Extra in future: set()
Treinando modelo para Canal: Own Stores | Corredor: PY
Missing in future: set()
Extra in future: set()
Treinando modelo para Canal: Own Stores | Corredor: MZ
Missing in fut

In [36]:
# Função para otimizar os hiperparâmetros
def optimize_xgboost(df, canal, corredor):
   print(f"Otimização para Canal: {canal} | Corredor: {corredor}")
   # Filtrar os dados para a combinação específica
   df_filtered = df[(df['Canal'] == canal) & (df['TOP_CORRIDOR'] == corredor)].copy()

   # Criar variáveis de tempo
   df_filtered['Date'] = pd.to_datetime(df_filtered['DTEE'])
   df_filtered.set_index('Date', inplace=True)

   # Separar variável-alvo
   y = df_filtered['TRANSACTIONS_TOTAL']

   # Definir o modelo base XGBoost
   forecaster = ForecasterRecursive(
       regressor=XGBRegressor(random_state=42),
       lags=7  # Temporário, ajustaremos depois
   )
   # Definir hiperparâmetros para testar
   param_grid = {
       'n_estimators': [100, 200, 300],
       'learning_rate': [0.01, 0.1, 0.3],
       'max_depth': [3, 5, 7],
       'subsample': [0.8, 1.0],
   }
   # Definir lags a serem testados
   lags_grid = [ [1, 3, 7], [1, 7, 14], [1, 7, 30], [1, 7, 30, 60] ]

   cv = TimeSeriesFold(
        steps              = 30,
        initial_train_size = len(y),
        refit              = True,
      )

   # Rodar Grid Search para encontrar os melhores hiperparâmetros
   results_grid = grid_search_forecaster(
       forecaster=forecaster,
       y=y,
       cv = cv,
       param_grid=param_grid,
       lags_grid=lags_grid,
       metric='mean_squared_error',
       verbose=False
   )

   # Selecionar os melhores hiperparâmetros
   best_params = results_grid.loc[results_grid['mean_squared_error'].idxmin(), :]
   print(f"Melhores parâmetros para {canal} - {corredor}: {best_params}")
   return best_params

# Aplicar a otimização para cada Canal e Corredor
optimized_models = []

for canal, corredor in itertools.product(df['Canal'].unique(), df['TOP_CORRIDOR'].unique()):
   best_params = optimize_xgboost(df, canal, corredor)
   if best_params is not None:
       optimized_models.append((canal, corredor, best_params))

# Exibir os melhores parâmetros encontrados para cada combinação
optimized_models_df = pd.DataFrame(optimized_models, columns=['Canal', 'Corredor', 'Melhores_Parametros'])
print(optimized_models_df)

Otimização para Canal: Own Stores | Corredor: OTHERS


lags grid:   0%|          | 0/4 [00:00<?, ?it/s]

params grid:   0%|          | 0/54 [00:00<?, ?it/s]

ValueError: If used, `initial_train_size` must be an integer greater than the `window_size` of the forecaster (7) and smaller than the length of `y` (86693).

In [10]:
df_predictions = pd.concat(all_predictions)

In [12]:
df_predictions['Predicted_transactions'].sum()

2014985.6

In [13]:
df_predictions.to_excel('/content/Check.xlsx')